### Extracting Data from pdf to dataframe

In [ ]:
import pandas as pd
import pdfplumber

In [ ]:
purchase_table_path = "/Users/saileshpanda/Desktop/IndiaAI/PDF/Purchase.pdf"
redemption_table_path = "/Users/saileshpanda/Desktop/IndiaAI/PDF/Redemption.pdf"

In [ ]:
flag = 0
with pdfplumber.open(purchase_table_path) as pdf:
    for page in pdf.pages:
        table = page.extract_tables()
        for pos,r in enumerate(table[0]):
            if flag == 0:
                purchase_table = pd.DataFrame(columns=r)
                flag = 1
            else:
                if pos==0:
                    continue
                purchase_table.loc[len(purchase_table)] = r

In [ ]:
purchase_table.tail(4)

In [ ]:
flag = 0
with pdfplumber.open(redemption_table_path) as pdf:
    for page in pdf.pages:
        table = page.extract_tables()
        for pos, r in enumerate(table[0]):
            if flag == 0:
                redemption_table = pd.DataFrame(columns=r)
                flag = 1
            else:
                if pos==0:
                    continue
                redemption_table.loc[len(redemption_table)] = r

In [ ]:
redemption_table.tail(4)

In [ ]:
redemption_table.set_index('Sr No.', inplace=True)
purchase_table.set_index('Sr No.', inplace=True)

### Doc Embedding

In [ ]:
from sentence_transformers import SentenceTransformer
import json

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
purchase_table['Embedding'] = None
for index, rows in purchase_table.iterrows():
    purchase_table["Embedding"][index] = embedding_model.encode(rows.to_csv())

In [ ]:
redemption_table['Embedding'] = None
for index, rows in redemption_table.iterrows():
    redemption_table["Embedding"][index] = embedding_model.encode(rows.to_csv())

In [ ]:
purchase_table.to_pickle("purchase_table.pkl")
redemption_table.to_pickle("redemption_table.pkl")

In [ ]:
purchase_table_pkl = pd.read_pickle("purchase_table.pkl")
redemption_table_pkl = pd.read_pickle("redemption_table.pkl")

#### Similarity

In [ ]:
import numpy as np
import ast
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def find_similar_doc(prompt, purchase_table, redemption_table):

    embedded_prompt = embedding_model.encode(prompt)

    purchase_similarity_score = []
    redemption_similarity_score = []

    for index, row in purchase_table.iterrows():
        similarity = cosine_similarity(embedded_prompt.reshape(1, -1), 
                                       row['Embedding'].reshape(1, -1))
        
        purchase_similarity_score.append((similarity, index))
    
    purchase_similarity_score = sorted(purchase_similarity_score, key=lambda x: x[0], reverse=True)
    
    for index, row in redemption_table.iterrows():
        similarity = cosine_similarity(embedded_prompt.reshape(1, -1), 
                                       row['Embedding'].reshape(1, -1))
        
        redemption_similarity_score.append((similarity, index))

    redemption_similarity_score = sorted(redemption_similarity_score, key=lambda x: x[0], reverse=True)
    
    return purchase_similarity_score, redemption_similarity_score

        

In [ ]:
prompt = "What is the total bond amount of A B C INDIA LIMITED"
p, r = find_similar_doc(prompt, purchase_table_pkl, redemption_table_pkl)

In [ ]:
p

### LLM

In [82]:
import google.generativeai as genai
import gradio as gr
import os

In [83]:
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel("gemini-pro")

In [89]:
prompt = purchase_table[:10]
prompt = prompt.to_csv()
prompt += " Prompt: Use the above csv file to extract the data asked in the below query. Query: What is the total bond amount enchased by TELUGU DESAM PARTY on 12th April 2019?"
response = model.generate_content(prompt)
print(response.text)

InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting

In [ ]:
import os
from groq import Groq

client = Groq(
    api_key=os.environ["GROQ_API_KEY"],
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

A fun challenge!

Given the provided CSV file, I'll help you extract the required data. Since the query asks for the total bond amount enchased by TELUGU DESAM PARTY on 12th April 2019, we need to filter the data based on two conditions:

1. The `Prefix` column should be "TL" (indicating TELUGU DESAM PARTY).
2. The `Journal Date` column should be "12/Apr/2019".

Here's the step-by-step process to extract the required data:

1. Use a CSV parser or a programming language like Python to read the CSV file.
2. Filter the data using the above conditions. You can use a `WHERE` clause in SQL or a conditional statement in Python to filter the data.
3. Extract the `Denominations` column values for the filtered data.
4. Calculate the total bond amount by summing up the values in the `Denominations` column.

Here's the Python code to accomplish this:
```python
import csv

with open('bond_data.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    filtered_data = [row for row in reader if 